In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import time
import dotenv
import os

dotenv.load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [72]:
dir = "./files/"
file_list = os.listdir(dir)

In [4]:
print(file_list)

['2025-AI-Trend-Report.pdf', 'Practice_document.pdf']


### Part1-1. 문서 기반 챗봇

In [5]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=300, 
                                               chunk_overlap=50, 
                                               separators=["\n\n"])

In [6]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[0])

In [7]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [10]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=split_docs)

In [11]:
# 리트리버 생성

retriever = vector_store.as_retriever()

In [8]:
prompt = ChatPromptTemplate([
    ("system", "문서 : {context}\n\n"
               "당신은 문서에서 원하는 정보를 찾는 10년차 데이터 전문가입니다. 항상 문서를 기반하여 응답해주세요\n"
               "문서에서 응답을 찾을 수 없는 경우 '문서에서 응답을 찾을 수 없습니다.' 라고 답변하세요."), 
    ("user", "{query}")

])

In [12]:
def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)

In [13]:
chain = {"context": retriever | RunnableLambda(format_docs),
         "query": RunnablePassthrough()} | prompt | llm

In [14]:
query = "2025년엔 어떤 부분을 준비해야할까?"

In [15]:
result = chain.invoke(query)

In [16]:
print(result.content)

2025년에는 다음과 같은 부분을 준비해야 할 것으로 보입니다:

1. **책임 있는 AI 운영**: 기업들은 윤리적이고 안전하며 투명한 AI 개발 프로세스를 구축해야 합니다. 이는 장기적인 성공을 위해 필수적인 요소로 여겨집니다.

2. **신뢰할 수 있는 AI**: 기업들은 신뢰할 수 있는 AI의 중요성을 인식하고, 이를 위한 다양한 전략을 모색해야 합니다. AI의 신뢰성을 높이기 위한 검증 서비스도 필요합니다.

3. **AI 기술의 정교화 및 전문화**: AI 기술이 더욱 정교해지고 전문화됨에 따라, 기업들은 이를 효과적으로 도입하고 활용할 수 있는 방법을 찾아야 합니다.

4. **지속 가능성과 사이버 보안**: AI 기술 활용에서 지속 가능성과 사이버 보안은 중요한 이슈로 떠오를 전망입니다. 관련 위험을 효과적으로 관리하는 노력이 필요합니다.

5. **에너지 효율성**: AI 시스템의 에너지 소비가 환경에 미치는 영향을 완화하기 위한 기술에 투자해야 합니다. 에너지 효율적인 칩, 혁신적인 냉각 기술, 탄소 배출 없는 에너지 원 등을 고려해야 합니다.

이러한 준비를 통해 기업들은 변화하는 환경에 적응하고 경쟁력을 높일 수 있을 것입니다.


In [61]:
result = chain.stream(query)

In [62]:
for step in result:
    print(step.content, sep="", end="")
    time.sleep(0.03)

2025년에는 다음과 같은 부분을 준비해야 할 것으로 보입니다:

1. **책임 있는 AI 운영**: 기업들은 윤리적이고 안전하며 투명한 AI 개발 프로세스를 구축해야 합니다. 이는 장기적인 성공을 위해 필수적인 요소로 여겨집니다.

2. **신뢰할 수 있는 AI**: 기업들은 신뢰할 수 있는 AI의 중요성을 인식하고, 이를 위한 다양한 전략을 모색해야 합니다. AI의 신뢰성을 높이기 위한 검증 서비스도 필요합니다.

3. **AI 기술의 정교화 및 전문화**: AI 기술이 더욱 정교해지고 전문화됨에 따라, 기업들은 이를 효과적으로 도입하고 활용할 수 있는 방법을 찾아야 합니다.

4. **지속 가능성과 사이버 보안**: AI 기술 활용에서 지속 가능성과 사이버 보안은 중요한 이슈로 떠오를 전망입니다. 관련 위험을 효과적으로 관리하는 노력이 필요합니다.

5. **에너지 효율성**: AI 시스템의 에너지 소비가 환경에 미치는 영향을 완화하기 위한 기술에 투자해야 합니다. 에너지 효율적인 칩, 혁신적인 냉각 기술, 탄소 배출 없는 에너지 원 등을 고려해야 합니다.

이러한 준비를 통해 기업들은 변화하는 환경에 적응하고 경쟁력을 높일 수 있을 것입니다.

In [63]:
result = chain.invoke("오늘 서울 날씨는 어때?")

In [64]:
print(result.content)

문서에서 응답을 찾을 수 없습니다.


In [67]:
result = chain.invoke("미국의 인구수는 몇 명정도 돼?")

In [68]:
print(result.content)

문서에서 응답을 찾을 수 없습니다.


### Part1-2 고객 응대 챗봇

In [73]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[1])

In [74]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [75]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=split_docs)

In [76]:
# 리트리버 생성

retriever = vector_store.as_retriever()

In [77]:
# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
    context : {context}

    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    ---
    ### 규칙 ###
    1. 주어진 context만을 이용하여 답변해야합니다. 
    2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    4. 항상 친절한 말투로 응대합니다.
    5. 하이퍼 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요. 아래 예시를 참고하여 서식을 맞추세요.
    **하이퍼퍼 링크 예시**
    5-1. [스타벅스 구역삼사거리점](https://naver.me/FV7K6xTM) 입니다.
    5-2. [화목순대국](https://naver.me/FQVGK6TZ) 입니다.
    5-3. [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5) 입니다.
    ---
    """),
    ("human", "{query}")
])

In [ ]:
chain = {"context": retriever | RunnableLambda(format_docs),
         "query": RunnablePassthrough()} | prompt | llm

In [79]:
query = "개강하는 날짜 알려주세요."

In [80]:
result = chain.invoke(query)

In [81]:
print(result.content)

아이펠 프로덕트 데이터 분석가 과정의 개강일은 2024년 12월 30일입니다. 더 궁금한 점이 있으시면 언제든지 문의해 주세요!


In [82]:
query = "오프라인으로도 수강할 수 있나요?"

In [83]:
result = chain.invoke(query)

In [84]:
print(result.content)

아이펠 교육과정은 100% 실시간 온라인 과정으로 진행되지만, 희망하시는 경우 오프라인 학습 공간인 [모두의연구소 강남](https://naver.me/FslREXGR) 또는 [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5)를 무료로 제공해드리고 있습니다. 공간 사용 방법은 입학 후 별도로 안내드릴 예정입니다.


In [88]:
query = "졸업하면 어떤걸 할 수 있나요?"

In [89]:
result = chain.invoke(query)

In [90]:
print(result.content)

졸업 후에는 데이터 분석가, PM/PO, 그로스마케터 등으로 취업할 수 있습니다. 더 궁금한 점이 있으시면 언제든지 문의해 주세요!


In [91]:
query = "서울에서 부산까지는 얼마나 걸리나요?"

In [92]:
result = chain.invoke(query)

In [93]:
print(result.content)

해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다.
